## Libraries

In [30]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.metrics import r2_score, mean_absolute_error
from pathlib import Path

In [31]:
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

In [32]:
%run ..//..//..//Library//SVRExtended_Library.ipynb 

## Load data

In [33]:
diabetes = load_diabetes()

# Initializing the dataframe
data = pd.DataFrame(diabetes.data)

#Adding the feature names to the dataframe
data.columns = diabetes.feature_names
data.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


In [34]:
#Adding target variable to dataframe
data['diabetes'] = diabetes.target 

In [35]:
# Spliting target variable and independent variables
X = data.drop(['diabetes'], axis = 1)
y = data['diabetes']

In [36]:
# Matrix with feature transformation
# X = pd.read_csv("..\\newBoston.csv")

In [37]:
# Splitting to training and testing data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 4)

In [38]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Optimization function

In [42]:
def opt_bas(C, epsilon, lamda, gamma):
    
    # parameters
    hyperparameters = {
        'kernel' : "rbf",
        'C' : C, 
        'epsilon' : epsilon, 
        'lamda' : lamda,
        'gamma' : gamma,
    }
    
    # fit and predict
    model = SVRExtended_cvxpy(**hyperparameters).fit(X_train, y_train)

    y_pred = model.predict(X_test)
    
    # get score
    metric = -r2_score(y_test, y_pred)
    
    return -metric

## Run optimization

In [43]:
class newJSONLogger(JSONLogger):

      def __init__(self, path):
            self._path=None
            super(JSONLogger, self).__init__()
            self._path = path if path[-5:] == ".json" else path + ".json"

In [53]:
# Bounded region of parameter space
pbounds = {'C': (120, 200), 'epsilon': (0.001, 1), 'lamda': (0.001, 1), 'gamma': (0.001, 0.08)}

# Bayes optimizer instantiation
optimizer = BayesianOptimization(f=opt_bas, 
                                 pbounds=pbounds, 
                                 random_state=3, verbose=2, 
                                )

# keep data
log_path = Path().resolve() / "Logs" / "Diabetes_r2_leo_cvxpy.json"
logger = newJSONLogger(path = str(log_path))
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [54]:
optimizer.maximize(init_points = 5, n_iter=500)

OSError: [Errno 22] Invalid argument